In [1]:
import numpy as np
import pandas as pd
import pickle

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
master_df = pd.read_pickle('data/master_df.pkl')
freq_df = pd.read_pickle('data/freq_df.pkl')
blacklist = open('data/blacklist').read().lower().split('\n')

In [3]:
freq_series = freq_df.max(axis=1)

In [4]:
chosen = ['sparris', 'pasta', 'tomater', 'lök']

In [5]:
# All recipes weighted by number of chosen
aa = master_df.ingredients.apply(lambda x: len(set(x).intersection(chosen)))
a = pd.Series([i for j in master_df.ingredients * aa for i in j]).value_counts()
a.divide(freq_series).dropna().sort_values(ascending=False).head(10)

bambuspett                         2.0
hela skalade tomater i tomatsås    2.0
italienska örter                   2.0
veggiebitar                        2.0
kastanjer                          2.0
kebabsås                           2.0
rimmat stekfläsk                   2.0
vongole                            1.5
spickeskinka                       1.5
vinblad                            1.5
dtype: float64

In [6]:
a = pd.Series([i for j in master_df[master_df.ingredients.apply(lambda x: any(i in x for i in chosen))].ingredients for i in j]).value_counts()
a.head(10)

salt           3124
tomater        2713
vitlök         1789
olivolja       1713
svartpeppar    1325
peppar         1286
gul lök         954
vatten          784
smör            779
citron          706
dtype: int64

In [7]:
min_count = 100
b = pd.Series()
for ingr, count in a.items():
    b[ingr]= count/freq_df.loc[ingr, ingr] * int(freq_series[ingr] >= min_count)
b.sort_values(ascending=False).head()

lök              0.972789
sparris          0.960714
tomater          0.915008
hamburgerbröd    0.666667
oliver           0.583051
dtype: float64

In [8]:
freq_df.loc[chosen].min().sort_values(ascending=False).head(10)

salt           253
varmrätt       177
olivolja       149
svartpeppar    116
peppar         107
koka            90
citron          83
smör            77
vitlök          76
i ugn           69
dtype: int64

In [9]:
bias = 10
freq_df.loc[chosen].min(axis=0).divide(freq_df.max(1) + bias).sort_values(ascending=False).head()

bondbönor              0.104167
matlagningsvin         0.068966
kronärtskockshjärta    0.050000
chilipasta             0.050000
sockerärtor            0.048387
dtype: float64

In [10]:
def suggest(chosen, bias=10):
    return freq_df.loc[chosen].min(axis=0).divide(freq_df.max(1) + bias).dropna().sort_values(ascending=False)

suggest(['lingon', 'potatis', 'grädde']).head(10)

lingon        0.390476
viltfärs      0.136364
messmör       0.130435
fläsk         0.111111
idealmjöl     0.107143
sugarsnaps    0.100000
kalvfärs      0.089286
älgfärs       0.087719
enbär         0.079646
skavkött      0.076923
dtype: float64